## Classiq for Qiskit Users

Welcome to the "Classiq for Qiskit Users" tutorial. This guide is designed for users already familiar with Qiskit and are looking to seamlessly transition to Classiq to build quantum algorithms with a higher-level functional design that enables developers to focus on the quantum program rather than a quantum circuit using the four pillars: Design, Optimize, Analyze, Execute.

<details markdown>
<summary markdown> New to Classiq? </summary>
If you're new to Classiq, we highly recommend you to check out our <a href="https://docs.classiq.io/latest/classiq_101/">Classiq 101</a> guide to get started!

#### Table of Contents

1. [Key Differences and Similarities between Qiskit and Classiq](#section1)
    - 1.1 Key Differences
        - 1.1.1 Design, Optimize, Analyze, Execute
            - 1.1.1.1 Design: Writing a Quantum Algorithm
            - 1.1.1.2 Optimize: The unique strength of Classiq
            - 1.1.1.3 Analysze: Visualization
            - 1.1.1.4 Analysis of Measurement Results
        - 1.1.2 Decomposition
    - 1.2 Similarities

2. [Building Blocks of Quantum Algorithms (with Simple Exercises)](#section2)
    - 2.1 [HELLO WORLD] Preparing the Initial State
        - 2.1.1 Exercise: Prepare a Minus State
    - 2.2 Constructing an Oracle:
        - 2.2.1 Exercise: Construct an Oracle such that $f(x)$
    - 2.3 Transpilation
    - 2.4 Changing Computational Basis
    - 2.5 Exporting Quantum Programs

3. [Quantum Algorithms](#section3)
    - 3.1 Phase Kickback
        - 3.1.1 Phase Kickback in Qiskit
        - 3.1.2 Phase Kickback in Classiq

4. [Appendix Notes](#section4)
    - Limitations: QEC and Mid Measurement


<a id="section1"></a> 1. Key Differences and Similarities between Qiskit and Classiq

##### 1.1 Key Differences 
Qiskit, as we know is designed to be a python SDK for quantum programming, which start with qubits and functions which aggregate these qubits with logical quantum gates to build quantum circuits. Classiq on the other hand offers an advantage of designing quantum algorithms without being tied to circuits. How? By using a higher-level functional design. In Classiq, quantum algorithms can be built using functions which contain native primitives and datatypes like QNum and QArray, which all then culminate in a main function. In addition to its python SDK, Classiq offers a native language, QMod which is how our quantum algorithm written in python connects with the easy-to-use visual Classiq platform for synthesis, optimization, analysis, and execution. This in turn also offers quantum experts flexible design and deep optimization of quantum programs for specific implementations.

Classiq has 4 components which form the pillar features enabling users to Design, Execute, Analyze, Execute quantum algorithms in a structured workflow which uses our high-level functional design

<b>Design</b>

The first step in quantum software development is to design your software and your algorithm. Classiq features a unique high-level modeling language called Qmod that naturally captures the core concepts of quantum algorithm design. There are two ways to design in Qmod: * Directly, via the Classiq IDE using the Qmod native syntax * With the Classiq Python SDK package, which gives access to the Qmod language via Python. Once you finish designing your algorithm, you send it to the Classiq synthesis engine (compiler) to create a concrete quantum circuit implementation - a quantum program.

<b>Optimize</b>

Classiq's unique strength is enabling intiutive optimization which is otherwise non-trivial using parameters like depth and width. There are two methods to perform optimization of your programs - One directly from the SDK using the special syntax given below in the Optimize section, and Two, on the Classiq platform's Synthesis configuration panel. 

<b>Analyze</b>

Once a quantum algorithm was designed and a quantum program was synthesized, the next natural thing one would to do is to analyze the quantum program and to check if the underlying quantum circuit implements the desired quantum algorithm properly and how. This can be done with the Classiq visualization tool that enables exploring the quantum circuit interactively on different level of functional hierarchies, as well as to extract crucial information about it.

<b>Execute</b>

The last step of the quantum algorithm development process with Classiq is to execute the quantum program on a quantum computer or a simulator. This can be done in the IDE or through the Python SDK. Classiq offers access to a wide variety of quantum computers with different hardware modalities from several companies including: IonQ, Quantinuum, IBM, OQC and Rigetti, as well as to several simulators. The execution phase is composed of configuration and access to the results.

##### 1.1.1 Design: Building a Quantum Algorithm

Let's explore the designing of a simple algorithm that performs an arithmetic operation. Consider the linear equation, $y = x^2 + 1$ and our task is to solve this equation with a quantum variable $|x\rangle$.

In [ ]:
from classiq import Output, QNum, allocate, hadamard_transform, qfunc

@qfunc
def main(x: Output[QNum], y: Output[QNum]):

    allocate(4, x)
    hadamard_transform(x)  # creates a uniform superposition
    y |= x**2 + 1

Note that any function written in Classiq, involves a decorator like <code>@qfunc</code> to be included before the start of the function definition. This is also the case for a main function. It's also noteworthy that any functions written in Classiq, can be fruitful only by assimilating them in a main function, as it is used for the synthesis of our quantum model.

The next step, would be the synthesis of a quantum model, which is a pre-cursor to the quantum program that can be then analyzed and executed on the platform.

In [ ]:
quantum_program = synthesize(create_model(main))
show(quantum_program)

After synthesis, we use <code>show()</code>, which calls the Classiq Platform API. This redirects us to the platform where we can view the synthesized quantum program.

##### 1.1.2 Optimize: Unique Strength of Classiq

While optimization using Qiskit is non-trivial, Classiq allows its users to optimize the depth, width, and gate count parameters with relative ease. In this section, we will discuss hands-on, the two methods for optimization of quantum programs in Classiq. It's not unknown that optimization using Qiskit would require deep domain knowledge in the art of advanced Optimization Techniques, which Classiq abstracts out for its users while still producing optimal results.

<b>Method 1: Directly from SDK</b>

Once we have written our program and ecncapsulated it in a main function for sythesis of a model, like described in the Design section, the following method can be used to give the parameter constraints specification:

In [ ]:
quantum_model = create_model(main)

Below, we build a new model with constraints from the model we have previosuly built and use <code>set_constraints()</code> to add the model and constraints, which is given within <code>Constraints()</code> function.

In [ ]:
quantum_model_with_constraints = set_constraints(
    quantum_model, Constraints(optimization_parameter="width", max_depth=500)
)

Now, we shall synthesize as usual but the upgraded model, using <code>synthesis(quantum_model_with_constraints)</code>

In [ ]:
quantum_program = synthesize(quantum_model_with_constraints)

Finally, we extract the parameters of the circuit implementation and print them out

In [ ]:
circuit_width = QuantumProgram.from_qprog(quantum_program).data.width
circuit_depth = QuantumProgram.from_qprog(quantum_program).transpiled_circuit.depth
print(f"The circuit width is {circuit_width} and the circuit_depth is {circuit_depth}")

<b>Method 2: Within the Platform</b>

<div style="text-align:center;">
    <img src="https://docs.classiq.io/resources/Shor_Algo_40Qubits_Only_Platform.mp4" alt="Optimize"></img>
</div>


In this example, we will demonstrate how we can optimize our quantum program for a maximum depth of 20 qubits. The quantum program in consideration here is our inbuilt Simon's Algorithm

##### 1.1.3 Analyze: Visualization

As humans, visualization helps us understand the intricacies of quantum programs better. Let's take the example program which uses 2 qubits to create the entangled bell state and visualize it on Qiskit and Classiq to understand the difference.

In [ ]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)

qc.draw('mpl')

While in Qiskit, <code>draw()</code> is used to visualize our quantum circuit, an interactive method to visualize is provided by our Classiq platform to expand into the intricacies of your quantum program.

Let's quicky write the equivalent circuit in Classiq SDK and explore how we can visualize it on the Classiq platform

In [ ]:
from classiq import *

@qfunc
def main(x: Output[QArray[QBit]]):
    prepare_bell_state(2, x)

qmod = create_model(main)

In [ ]:
write_qmod(qmod, "prepare_bell_state")
qprog = synthesize(qmod)
show(qprog)

After synthesis, we use <code>show()</code>, which calls the Classiq Platform API and redirects to the platform where we can view and analyze it using the expand and contract buttons denoted by $+$ and $-$ symbols respectively.

<div style="text-align:center;">
    <img src="https://docs.classiq.io/resources/analyze_cfqu.mp4" alt="Analyze"></img>
</div>

##### 1.1.4 Decomposition

Decomposition is the method of scientifically breaking down a given quantum program into smaller computational operations or components. This is useful for investigation of quantum programs as well as for optmization techniques for specific quantum implementations. Let's now see how we can perform decomposition in Qiskit and how it's possible through Classiq in just a click of an interactive button! We will consider the Quantum Fourier Transform to understand hands-on how to perform decomposition.

In [ ]:
# Decomposition in Qiskit

from numpy import pi

def qft_rotations(circuit, n):
    if n == 0: # Exit function if circuit is empty
        return circuit
    n -= 1 # Indexes start from 0
    circuit.h(n) # Apply the H-gate to the most significant qubit
    for qubit in range(n):
        # For each less significant qubit, we need to do a
        # smaller-angled controlled rotation: 
        circuit.cp(pi/2**(n-qubit), qubit, n)

qc = QuantumCircuit(4)
qft_rotations(qc,4)
qc.draw()

decomposed_circuit = qc.decompose()
print("Decomposed QFT Circuit:")
decomposed_circuit.draw(output='mpl').show()

<div style="text-align:center;">
    <img src="https://docs.classiq.io/resources/decomp_qft.mp4" alt="Decomposition"></img>
</div>

##### 1.1.5 Analysis of Measurement Results

One of the most useful analysis tools with visualization used in quantum computing is the histogram to see observe and analyse the measurement results. While Qiskit has a separate sub-package qiskit.visualization, part of their SDK, Classiq has a seamless Jobs page which is automatically show after the user executes their quantum program. Let's explore how we can arrive at a measurement result in the histogram form with Qiskit and Classiq. The example program we'll use here again is the preparation of Bell State

##### 1.2 Similarities

One of the key similarities between Qiskit and Classiq lies in how we apply unitary operations or gates on qubits, although within the construct of Qiskit, gates are applied on qubits which are part of a circuit whereas in Classiq, it's a higher level function. Let's look at a simple example:

In [ ]:
# Qiskit 

qc = QuantumCircuit(1)
qc.h(0) # applying $H$ on the qubit
qc.y(0)

In [14]:
# Classiq

@qfunc
def operation(out: Output[QBit]):
    allocate(1, out) # initializing one qubit
    H(out)
    Y(out)

# To test this code, you may write a main function

<a id="section2"></a>2. Building Blocks of Quantum Algorithms

In this section of the tutorial, we will explore some building blocks of quantum computing, some of which will act as tools for building quantum algorithms discussed in the next section. We will also do some simple and fun exercises to get a hands-on understanding of how to program with Classsiq SDK!

##### 2.1 HELLO WORLD: Preparing the initial state

Let's prepare the $|+\rangle$ state using Qiskit and see how the process is different with Classiq:

In [ ]:

from qiskit import QuantumCircuit

# Create a quantum circuit with 1 qubit
qc = QuantumCircuit(1)

# Initialize the qubit to the |+> state
qc.h(0)


In [ ]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(1)
qc.h(0)

What we do here in qiskit is to apply the Hadamard gate <code>(qc.h(0))</code> to the qubit which we know creates a superposition of $|0\rangle$ and $|1\rangle$ states, resulting in the $|+\rangle$ state. Now, let's see how to prepare this state using Classiq.

In [ ]:
from classiq import *

@qfunc
def prepare_plus_state(out: Output[QBit]):
    allocate(1, out)
    H(out)

This gives us an advantage of abstraction from using a lower level circuit entities as our building blocks and allows for repeated usage of this state preparation at various points in your quantum program.

##### 2.1.1 Exercise

Write a function that prepares the minus state $\ket{-}=\frac{1}{\sqrt2}(\ket{0}-\ket{1})$, assuming it recives the qubit $\ket{x}=\ket{0}$

<details>
<summary>
HINT
</summary>

Use `H(x)`,`X(x)`
</details>

Solution:

In [ ]:
@qfunc
def prepare_minus_state(x:QBit):
    X(x)
    H(x)

We can now test our code by calling the state preparation function without a main function:

In [ ]:
@qfunc
def main(x: Output[QBit]):
    allocate(1,x) # Initalize the qubit x
    prepare_minus_state(x) # Prepare the minus state

In [ ]:
quantum_model = create_model(main)
quantum_program = synthesize(quantum_model)

In [ ]:
show(quantum_program)

Good job with the exercise! Let's continue.

##### 2.2 Constructing an Oracle

Often, quantum algorithms are built around investigating a given function $f(x)$ where we assume the function to be a black box that receives $x$ as input and gives $f(x)$ as its corresponding output. We will now explore how we can construct such oracles with Classiq in contrast to Qiskit.

**Detailed Example:** 

Let's assume that we want to build an oracle to determine whether a function $f(x)$ is balanced or constant. This is an interesting oracle which is the building block of the popular Deutsch-Jozsa Algorithm. Our predicate to check if the function is balanced or constant would be defined within a function <code>simple_predicate</code>.

Consider the predicate <code>res ^= x > 7</code>

This means that 
* whenever x > 7 is True, it evaluates 1 and res is XORed with 1, effectively toggling it to 0 or 1. 
* whenever x > 7 is False, res is XORed with 0, effectively always leaving its value unchanged

In simple words, this predicate is the question we want our oracle to answer. As you can clearly see in this example, the first scenario corresponds to the function being balanced and the latter to being constant. 

Observe keenly the <code>oracle</code> function in the below code snippet. We can see that the quantum circuit <code>qc</code> and the oracle itself are not inseparable, creating a dependency and lower-level design, which requires the oracle to be fed in with the quantum circuit, the function input, as well as an additional qubit for condition checking purposes.

In [ ]:
from qiskit import QuantumCircuit, Aer, execute

# Function oracle: f(x) = 1 if x > 7, otherwise f(x) = 0
def oracle(circuit, x, ancilla):
    for i in range(len(x)):
        circuit.cx(x[i], ancilla)
    # For x > 7, flip the ancilla qubit
    circuit.x(ancilla).c_if(x, 7)


In a stark constrast to the above code, Classiq simplifies the process by allowing the preparation of our initial state, independent declaration of the predicate that the oracle needs to check, and also leverages an in-built function <code>within_apply</code> to write mechanism of the oracle in a powerful way.

In [ ]:
# Oracle for Deutsch Jozsa Algorithm in Classiq

@qfunc
def apply_oracle(x: QNum):
    aux = QBit("aux")
    within_apply(
        compute=lambda: prepare_minus(aux), action=lambda: constant_function(x, aux)
    )

In [ ]:
# Complete Working Example

from classiq import *

@qfunc
def constant_function(x: QNum, res: QBit):
    res ^= (x > 7)


@qfunc
def prepare_minus(out: Output[QBit]):
    allocate(1, out)
    X(out)
    H(out) 


@qfunc
def apply_oracle(x: QNum):
    aux = QBit("aux")
    within_apply(
        compute=lambda: prepare_minus(aux), action=lambda: constant_function(x, aux)
    )

@qfunc
def main(x: Output[QNum]):
    allocate(4, x)
    within_apply(compute=lambda: hadamard_transform(x), action=lambda: apply_oracle(x))

quantum_model = create_model(main)
quantum_program = synthesize(quantum_model)
show(quantum_program)

##### 2.3 Changing Computational Basis

We will explore how we can work with compuational bases and change them for specific applications in this section of the tutorial with the example of Variational Quantum Eigensolver (VQE) algorithm. It's expert knowledge that VQE consists of rotational pauli and controlled-X operations. Qiskit enables us to use their transpilation tool to mention the ansatz circuit to use specified basis gates, in out example, <code>["rx", "ry", "rz", "cx", "cy"]</code>

In [ ]:
from qiskit import QuantumCircuit, Aer, transpile, execute
from qiskit.circuit.library import TwoLocal
from qiskit.algorithms import VQE
from qiskit.primitives import Estimator
from qiskit.opflow import X, Z, I
from qiskit.algorithms.optimizers import COBYLA

# Define the Hamiltonian for a simple problem (e.g., H = X + Z)
hamiltonian = X + Z

# Define the ansatz (a simple two-local ansatz)
ansatz = TwoLocal(rotation_blocks=['ry', 'rz'], entanglement_blocks='cx', reps=1)

# Set up the VQE instance
vqe = VQE(ansatz, optimizer=COBYLA(), estimator=Estimator(), quantum_instance=Aer.get_backend('statevector_simulator'))

# Execute VQE to find the ground state energy
result = vqe.compute_minimum_eigenvalue(operator=hamiltonian)

# Get the resulting quantum circuit
circ = vqe.ansatz

# Transpile the circuit to the specified basis gates
transpiled_circ = transpile(circ, basis_gates=["rx", "ry", "rz", "cx", "cy"])

# Print the transpiled circuit
print(transpiled_circ)

<div style="text-align:center;">
    <img src="https://docs.classiq.io/resources/comp_basis" alt="Basis Change"></img>
</div>

##### 2.4 Exporting Quantum Programs

QASM, or Quantum Assembly Language serves as one of the most intricate ways to export quantum programs from one SDK to other without hassle. Qiskit and Classiq both provide methods to export your quantum program using QASM but Classiq additionally offers other formats like LaTeX, HTML, JPEG, JSON, and transpiled QASM for other research purposes in addition to QASM enabling cross-SDk collaborations. Classiq provides an interactive button which enables users to swiftly export their programs.

Exporting a quantum circuit from Qiskit as QASM file

In [ ]:
from qiskit import QuantumCircuit
from qiskit.qasm2 import dumps # For exporting as a QASM string for further processing
from qiskit.qasm2 import dump # For exporting as a QASM file for cross-SDK usage
 
qc = QuantumCircuit(1)
 
qasm_str = dumps(qc)

with open("my_quantum_program.qasm", "w") as f:
    dump(qc, f)

<a id="section3"></a> 3. Quantum Algorithms: Phase Kickback

Phase Kickback is quantum primitive of high importance, used in larger quantum algorithms, e.g., Shor’s, Simon’s, Deutsch-Josza, and Grover’s. It deals with kicking the result of a function to the phase of a quantum state so it can be smartly manipulated with constructive and destructive interferences to achieve the desired result. This means that a phase shift applied to a control qubit during a controlled operation is "kicked back" to the control qubit.

3.1 Phase Kickback with Qiskit

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, transpile, execute
from qiskit.visualization import plot_histogram

# Define the number of qubits for the input
num_qubits = 4

# Create quantum and classical registers
x = QuantumRegister(num_qubits, 'x')
target = QuantumRegister(1, 'target')
c = ClassicalRegister(num_qubits, 'c')
qc = QuantumCircuit(x, target, c)

# Apply Hadamard transform to the x qubits
qc.h(x)

# Prepare the |-> state in the target qubit
qc.x(target)
qc.h(target)

# Oracle function: target ^= (x == 0)
# Apply X gates to flip x qubits for zero state detection
for i in range(num_qubits):
    qc.x(x[i])

# Apply a multi-controlled Toffoli gate (mct)
qc.mct(x[:], target)

# Apply X gates again to flip x qubits back
for i in range(num_qubits):
    qc.x(x[i])

# Measure the x qubits
qc.measure(x, c)

# Simulate the circuit
backend = Aer.get_backend('qasm_simulator')
transpiled_qc = transpile(qc, backend)
result = execute(transpiled_qc, backend, shots=1024).result()
counts = result.get_counts()

# Display the circuit and result
print(qc)
plot_histogram(counts).show()

On the higher level, we can observe from this code snippet that the logic depends heavily on applying the right gates and the core of the algorithm lies in flipping qubits based on the condition <code>target ^= (x == 0)</code> which we use an oracle to check for. Let's dig in deeper.

Firstly, we initialize x registers, which act as the target and control qubits manually. Then, the hadamard gate is applied to these x qubits indicating a hadamard transform. The target qubit is then utilized to prepare a $|-\rangle$ state. The purpose of the oracle here is to check if the state of each of these qubits is $|0\rangle$ using a fliping mechanism using $pauli X$ gate for the zero state detection. This is followed by the usage of the multi-controlled Toffoli gate which flips the target qubit only when all control qubits are in a specific state, in this case $|0\rangle$ after which the x qubits are flipped back before measurement.

3.2 Phase Kickback with Classiq

In [ ]:
from classiq import (
    H,
    Output,
    QBit,
    QNum,
    X,
    allocate,
    create_model,
    hadamard_transform,
    qfunc,
    show,
    synthesize,
    within_apply,
    write_qmod,
)


@qfunc
def prepare_minus(target: Output[QBit]):
    allocate(out=target, num_qubits=1)
    X(target)
    H(target)


@qfunc
def oracle_function(target: QBit, x: QNum):
    target ^= x == 0


@qfunc
def oracle_phase_kickback(x: QNum):
    target = QBit("target")
    within_apply(
        compute=lambda: prepare_minus(target), action=lambda: oracle_function(target, x)
    )


@qfunc
def main(x: Output[QNum]):
    allocate(num_qubits=4, out=x)
    hadamard_transform(x)
    oracle_phase_kickback(x)


qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

write_qmod(qmod, "phase_kickback")

Immediately upon viewing the above code snippet, we can realize stark differences from the equivalent code in qiskit. Let's explore them one by one.

The power of functional design allows us to build a function <code>prepare_minus_state</code> which encapsulates the operation of preparing a $|-\rangle$ using the pauli $X$ and hadamard $H$ gates, is the fact that it's reusable anywhere in your program later. It's nonteworthy that phase kickback is itself a primitive used within larger quantum programs, meaning resusablitily of functions can enable better software engineering practices and potentially, efficiency of your code.

<a id="section4"></a> 4. Appendix Notes

<b>Mid Circuit Measurement</b>


While mid-circuit measurement is a feature currenntly offered by Qiskit, Classiq plans to include this particular feature in its later releases as this feature is known to help researchers optimize algorithms, reduce the number of required qubits, and even enable new types of computations. 

<b>Noise Models and Error Correction</b>

Classiq currently does not offer simulation of noise models and thus, the tool doesn't contain error correction mechanisms as well.

#### References

[1] Qiskit Textbook <a href="(https://github.com/Qiskit/textbook/tree/main/notebooks/ch-demos#)">(GitHub)</a>